In [61]:
import pandas as pd
# for text processing
from unidecode import unidecode
import re
from nltk import ngrams
# for data modeling
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
# for plotting
from plotly import graph_objects as go
import plotly.express as px


## Load Data

In [2]:
data = pd.read_excel('../data/Projetos_de_Lei_2020_COVID.xlsx')
data

,nome,inteiro_teor_url,situacao,autor_nome,autor_partido,autor_sigla_uf,data_apresentacao,ementa,url
0,PL 1393/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Evair Vieira de Melo,PP,ES,01/04/2020,Estabelece medidas excepcionais de proteção so...,https://www.camara.leg.br/proposicoesWeb/ficha...
1,PL 765/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Reginaldo Lopes,PT,MG,19/03/2020,Autoriza o Poder Executivo a conceder perdão d...,https://www.camara.leg.br/proposicoesWeb/ficha...
2,PL 1324/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,José Nelto,PODE,GO,31/03/2020,Dispõe sobre dispensação de prescrição de rece...,https://www.camara.leg.br/proposicoesWeb/ficha...
3,PL 852/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Kim Kataguiri,DEM,SP,23/03/2020,Institui a suspensão do pagamento de jetons em...,https://www.camara.leg.br/proposicoesWeb/ficha...
4,PL 1212/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Ted Conti,PSB,ES,30/03/2020,Dispõe sobre a utilização extraordinária do Fu...,https://www.camara.leg.br/proposicoesWeb/ficha...
5,PL 993/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Benedita da Silva,PT,RJ,25/03/2020,Dispõe sobre regras aplicáveis ao trabalho dom...,https://www.camara.leg.br/proposicoesWeb/ficha...
6,PL 836/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Ruy Carneiro,PSDB,PB,23/03/2020,Suspende a cobrança de juros do cartão de créd...,https://www.camara.leg.br/proposicoesWeb/ficha...
7,PL 832/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,José Ricardo,PT,AM,23/03/2020,"Altera a Lei nº 13.979 de 2020 para instituir,...",https://www.camara.leg.br/proposicoesWeb/ficha...
8,PL 827/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,André Janones,AVANTE,MG,23/03/2020,Suspende pelo período de 90 (noventa) dias em ...,https://www.camara.leg.br/proposicoesWeb/ficha...
9,PL 661/2020,https://www.camara.leg.br/proposicoesWeb/prop_...,Aguardando Despacho do Presidente da Câmara do...,Efraim Filho,DEM,PB,17/03/2020,Institui medidas de proteção ao emprego e ao e...,https://www.camara.leg.br/proposicoesWeb/ficha...


In [130]:
def clean_text(text):
    text = text.lower()
    text = unidecode(text)
    text = text.replace('-', '')
    text = text.replace('/', '_')
    text = re.sub('\s+', ' ', text)
    text = re.findall('\w+', text)
    text = ' '.join(text)
    return text

def add_br(text):
    new_text = ''
    i = 0
    for word in text.split(' '):
        if i == 10:
            new_text += " " + word + '<br>'
            i = 0
        else:
            new_text += " " + word
            i += 1
    return new_text

## Clean text and Tf-Idf vectorize it

In [131]:
corpus = data['clean_ementa']
vectorizer = TfidfVectorizer(ngram_range=(1,2))
x = vectorizer.fit_transform(corpus)
#vectorizer.get_feature_names()
x

<352x6767 sparse matrix of type '<class 'numpy.float64'>'
	with 21268 stored elements in Compressed Sparse Row format>

## Apply dimensionality Reducion with Single Value Decompositon
The goal is to reduce the 6767 dimens'ions to 4, so we can visualize it.

In [139]:
svd = TruncatedSVD(n_components=4, n_iter=100)

principalComponents = svd.fit_transform(x)

principal_df = pd.DataFrame(data = principalComponents, columns = ['pc_1', 'pc_2', 'pc_3', 'color'])
principal_df['ementa_labels']= data['clean_ementa'].apply(add_br)
principal_df['autor'] = data['autor_nome']
principal_df['sg_partido'] = data['autor_partido']
print(svd.explained_variance_)

[0.01040533 0.01436544 0.0122729  0.01090822]


## Plot

In [140]:
fig = px.scatter_3d(principal_df,
                 x='pc_1',
                 y='pc_2',
                 z='pc_3',
                 color='color',
                 hover_data=['autor', 'sg_partido', 'ementa_labels'])
fig.show()